In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input, Flatten, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [3]:
input = Input(shape=(200,200, 3),name = 'image_input')

output_vgg16_conv = model_vgg16_conv(input)

x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(5, activation='softmax', name='predictions')(x)

my_model = Model(inputs=input, outputs=x)

my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 200, 200, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
predictions (Dense)          (None, 5)                 92165     
Total params: 14,806,853
Trainable params: 14,806,853
Non-trainable params: 0
_________________________________________________________________


In [4]:
#Generate datasets with keras library

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (200, 200),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (200, 200),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 3485 images belonging to 5 classes.
Found 838 images belonging to 5 classes.


In [5]:
classes = training_set.class_indices
print (classes)

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [6]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

In [7]:
my_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

my_model.fit_generator(training_set,
                         steps_per_epoch = 108,
                         epochs = 50,
                         validation_steps = 26,
                         validation_data = test_set,
                         callbacks=[reduce_lr])

Epoch 1/50
108/108 [==============================] - 62s 573ms/step - loss: 1.6259 - acc: 0.2989 - val_loss: 1.3469 - val_acc: 0.3846
Epoch 2/50
108/108 [==============================] - 62s 570ms/step - loss: 1.2396 - acc: 0.4703 - val_loss: 1.2365 - val_acc: 0.4938
Epoch 3/50
108/108 [==============================] - 58s 533ms/step - loss: 1.1784 - acc: 0.5182 - val_loss: 1.1133 - val_acc: 0.5261
Epoch 4/50
108/108 [==============================] - 58s 536ms/step - loss: 1.0969 - acc: 0.5551 - val_loss: 1.1999 - val_acc: 0.5310
Epoch 5/50
108/108 [==============================] - 59s 547ms/step - loss: 1.0784 - acc: 0.5692 - val_loss: 1.0620 - val_acc: 0.5583
Epoch 6/50
108/108 [==============================] - 59s 547ms/step - loss: 1.0594 - acc: 0.5817 - val_loss: 1.1312 - val_acc: 0.5087
Epoch 7/50
108/108 [==============================] - 59s 546ms/step - loss: 1.0245 - acc: 0.6006 - val_loss: 1.0430 - val_acc: 0.5682
Epoch 8/50
108/108 [==============================] - 5

In [8]:
my_model_json = my_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(my_model_json)
# serialize weights to HDF5
my_model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
